In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from random import choice

#### Projeto 2 - Organizando uma eleição

In [5]:
#cadastrar todos os apartamentos primeiro
class Apartamento:
    def __init__(self,numero_apartamento):
        self._lista_moradores = []
        self._status_votacao = False
        self._numero_apartamento = numero_apartamento
    
    def __repr__(self):
        return f'Nº Apartamento: {self._numero_apartamento},Moradores: {self._lista_moradores}, Votou?: {self._status_votacao}'

    def adiciona_morador(self,morador):
        if(isinstance(morador,Morador)):
            self._lista_moradores.append(morador)
            morador._apartamento = self
        else:
            print("Morador inválido. Não é um objeto Morador")
    
    def lista_moradores(self):
        lista_moradores = [{"Apartamento "+str(self._numero_apartamento): morador._nome} for morador in self._lista_moradores]
        df = pd.DataFrame(lista_moradores)
        display(df) 

class Morador:
    def __init__(self, nome):
        if(isinstance(nome,str)):
            self._nome = nome
            self._apartamento = None
        else:
            print("O nome deve ser uma string")

      
    def __repr__(self):
        return f'Morador: {self._nome}, Apartamento: {self._apartamento}'
    
    def vota_candidato(self,urna,numero_candidato):
        if(isinstance(urna,Urna)):
            if(isinstance(numero_candidato,int)):
                if self._apartamento._status_votacao:
                    print("Voto do apartamento já contabilizado. Não é possível voltar 2 vezes.")
                    return False
                else:
                    return urna.executa_voto(self._apartamento,numero_candidato)
            else:
                print("Nº do candidato inválido.")
        else:
            print("Urna inválida. Não é um objeto Urna")
                
class Candidato(Morador):
    def __init__(self,morador):
        if(isinstance(morador,Morador)):
            self._nome = morador._nome
            self._apartamento = morador._apartamento
            self._numero_urna = 0
            self._contagem_votos = 0
        else:
            print("Morador inválido. Não é um objeto Morador")

    def __repr__(self):
        return f'Candidato: {self._nome}, Número Urna: {self._numero_urna}'

    def atualiza_votos(self):
        self._contagem_votos = self._contagem_votos + 1
    
    @property
    def get_numero_urna(self):
        return self.__numero_urna
    
    @property
    def get_contagem_votos(self):
        return self.__contagem_votos
    
class Urna:
    def __init__(self):
        self._lista_candidatos = dict()
        self._lista_apartamentos = []
        
    def cadastra_apartamento(self,apartamento):
        if(isinstance(apartamento,Apartamento)):
            self._lista_apartamentos.append(apartamento)
        else:
            print("Apartamento inválido. Não é um objeto Apartamento")
            
    def cadastra_candidato(self,candidato):
        if(isinstance(candidato,Candidato)):
            numero_candidato = choice([i for i in range(1,1000) if i not in self._lista_candidatos.keys()])
            candidato._numero_urna = numero_candidato
            self._lista_candidatos.update({numero_candidato:candidato})
        else:
            print("Candidato inválido. Não é um objeto Candidato")
            
    def executa_voto(self,apartamento,numero_candidato):
            if not isinstance(numero_candidato,int):
                print("Dado inválido. Deve ser fornecido um número")
            elif not isinstance(apartamento,Apartamento):
                print("Apartamento inválido. Não é um objeto Apartamento")
            elif numero_candidato not in self._lista_candidatos.keys():
                print("Nº de candidato não registrado.")
            elif apartamento not in self._lista_apartamentos:
                print("Apartamento não habilitado para votar.")
            else:
                self._lista_candidatos[numero_candidato].atualiza_votos()
                apartamento._status_votacao = True
                self._lista_apartamentos.remove(apartamento)
                print(self._lista_apartamentos)
                if(len(self._lista_apartamentos) == 0):
                    return True
            return False
        
    def imprime_resultado(self):
        ranking = [[candidato._nome,candidato._contagem_votos] for candidato in self._lista_candidatos.values()]
        ranking = sorted(ranking, key=lambda x: x[1],reverse=True)
        df = pd.DataFrame(ranking)
        display(df)
        
    def mostra_candidatos(self):
        lista_candidato = [{"Candidato": candidato._nome,"Nº Urna": key} for key,candidato in self._lista_candidatos.items()]
        df = pd.DataFrame(lista_candidato)
        display(df)

In [6]:
moradores_do_predio = pd.read_csv('moradores.csv',';')  
#moradores_do_predio = pd.DataFrame(moradores_do_predio)
display(moradores_do_predio)


,Nome do Morador,Apartamento,Candidato
0,Joyce,101,True
1,Matheus,101,False
2,Rafael,101,False
3,Luis,102,True
4,Camila,201,False
5,André,201,False
6,João,202,False


In [7]:
#lê arquivo csv de apartamentos
apartamentos = pd.read_csv('apartamentos.csv')

#cria a variavel que vai guardar os apartamentos
apartamentos_existentes = {}

for apartamento in apartamentos.values:
    apartamentos_existentes[apartamento[0]] = Apartamento(apartamento[0])

In [8]:
#cria moradores e candidatos
moradores_existentes = []
candidatos_existentes = []

for morador in moradores_do_predio.values:
    novo_morador = Morador(morador[0])
    moradores_existentes.append(novo_morador)
    apartamentos_existentes[morador[1]].adiciona_morador(novo_morador)
    #verifica se o morador é candidato
    if morador[2]:
        candidatos_existentes.append(Candidato(novo_morador))

In [9]:
#lista moradores de cada apartamento
for apartamento in apartamentos_existentes.values():
    apartamento.lista_moradores()

,Apartamento 101
0,Joyce
1,Matheus
2,Rafael


,Apartamento 102
0,Luis


,Apartamento 201
0,Camila
1,André


,Apartamento 202
0,João


In [10]:
#lista os candidatos
lista_candidato = [{"Candidato": candidato._nome,"Nº Urna": candidato._numero_urna} for candidato in candidatos_existentes]
df = pd.DataFrame(lista_candidato)
display(df)

,Candidato,Nº Urna
0,Joyce,0
1,Luis,0


In [11]:
#criar urna
urna  = Urna()

In [12]:
#cadastrar apartamentos na urna
for apartamento in apartamentos_existentes.values():
    #verifica se apartamento não está vazio
    if len(apartamento._lista_moradores) > 0:
        urna.cadastra_apartamento(apartamento)

In [13]:
#cadastrar candidatos
for candidato in candidatos_existentes:
    urna.cadastra_candidato(candidato)

In [14]:
urna.mostra_candidatos()

,Candidato,Nº Urna
0,Joyce,321
1,Luis,77


In [15]:
terminou = False
display(moradores_do_predio)
while terminou == False:
    quemVaiVotar = int(input("Digite o id de qual morador vai votar: "))
    print()
    print("Olá ",moradores_existentes[quemVaiVotar]._nome)
    urna.mostra_candidatos()
    seuCandidato = int(input("Digite o número do seu candidato."))
    print()
    terminou = moradores_existentes[quemVaiVotar].vota_candidato(urna,seuCandidato) 
urna.imprime_resultado()

,Nome do Morador,Apartamento,Candidato
0,Joyce,101,True
1,Matheus,101,False
2,Rafael,101,False
3,Luis,102,True
4,Camila,201,False
5,André,201,False
6,João,202,False


Digite o id de qual morador vai votar: 0

Olá  Joyce


,Candidato,Nº Urna
0,Joyce,321
1,Luis,77


Digite o número do seu candidato.77

[Nº Apartamento: 102,Moradores: [Morador: Luis, Apartamento: Nº Apartamento: 102,Moradores: [...], Votou?: False], Votou?: False, Nº Apartamento: 201,Moradores: [Morador: Camila, Apartamento: Nº Apartamento: 201,Moradores: [...], Votou?: False, Morador: André, Apartamento: Nº Apartamento: 201,Moradores: [...], Votou?: False], Votou?: False, Nº Apartamento: 202,Moradores: [Morador: João, Apartamento: Nº Apartamento: 202,Moradores: [...], Votou?: False], Votou?: False]
Digite o id de qual morador vai votar: 1

Olá  Matheus


,Candidato,Nº Urna
0,Joyce,321
1,Luis,77


...

,Candidato,Nº Urna
0,Joyce,321
1,Luis,77


Digite o número do seu candidato.321

[Nº Apartamento: 202,Moradores: [Morador: João, Apartamento: Nº Apartamento: 202,Moradores: [...], Votou?: False], Votou?: False]
Digite o id de qual morador vai votar: 6

Olá  João


,Candidato,Nº Urna
0,Joyce,321
1,Luis,77


Digite o número do seu candidato.321

[]


,0,1
0,Joyce,2
1,Luis,2


In [16]:
urna.imprime_resultado()



def graficoVotos(candidatos, votos):
    plt.bar(candidatos, votos)
    plt.title('Resultado das eleições')
    plt.xlabel('Candidatos')
    plt.ylabel('Votos')

,0,1
0,Joyce,2
1,Luis,2
